In [1]:
import numpy as np
import pandas as pd 
import torch

In [22]:
train = pd.read_csv('train.tsv',sep='\t')
test = pd.read_csv('test.tsv',sep='\t')
train

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
...,...,...,...,...
156055,156056,8544,Hearst 's,2
156056,156057,8544,forced avuncular chortles,1
156057,156058,8544,avuncular chortles,3
156058,156059,8544,avuncular,2


In [23]:
print(train['Phrase'].str.len().max())
print(test['Phrase'].str.len().max())

283
256.0


In [69]:
from collections import Counter
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset

class MovieReviewsDataset(Dataset):
    def __init__(self, X, y, word2idx=None):
        super(MovieReviewsDataset, self).__init__()
        self.word2idx = word2idx
        self.X = X.map(lambda x: x.split())
        self.y = y

    def __len__(self):
        return self.y.shape[0]

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def preprocess(self):
        # TODO: add from nltk.tokenize import word_tokenize
        self.X = self.X.map(lambda x: x.split())
        # TODO: remove stop-words
        if self.word2idx is None:
            words = [word for row in self.X for word in row]
            vocab = Counter(words)
            vocab = sorted(vocab, key=vocab.get, reverse=True)
            self.word2idx = {word: ind + 2 for ind, word in enumerate(vocab)} 
            self.word2idx['[PAD]'] = 0
            self.word2idx['[UNKNOWN]'] = 1
            print(len(self.word2idx))
        
        self.X = list(map(lambda x: [self.word2idx[elem] if elem in self.word2idx else 1 for elem in x], self.X))
        print(f'max_len = {max([len(elem) for elem in self.X])}')
        #t_inp = torch.tensor(t_inp, dtype=torch.int32)
        self.X = list(map(lambda x: torch.tensor(x, dtype=torch.int32), self.X))
        self.X = pad_sequence(self.X, batch_first=True)
        #self.X = self.X.reshape(())
        #return self.X

# TODO: add torch.utils.data.Dataset

train = MovieReviewsDataset()
preprocess(train, word2idx).shape

0         [A, series, of, escapades, demonstrating, the,...
1         [A, series, of, escapades, demonstrating, the,...
2                                               [A, series]
3                                                       [A]
4                                                  [series]
                                ...                        
156055                                         [Hearst, 's]
156056                        [forced, avuncular, chortles]
156057                                [avuncular, chortles]
156058                                          [avuncular]
156059                                           [chortles]
Name: Phrase, Length: 156060, dtype: object
18228
max_len = 52
52


torch.Size([156060, 52])